In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
#import tensorflow as tf
import numpy as np
import glob
import os
import sys  
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing


from keras.models import Sequential
from keras.layers import *
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
#import tensorflow as tf; print(tf.__version__)
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
set_session(tf.Session(config=config))

In [3]:
data_path="npz_data/ucf_classes/224/*"
img_size=(224,224,3)
frame_number=20
num_class=2

In [4]:
def load_data(data_path):
    data=np.empty([1,frame_number,img_size[0],img_size[1],3])
    label=np.empty([1,])
    file_list=glob.glob(data_path)
    data_list=[d for d in file_list if "data" in d.split("/")[-1]]
    label_list=[l for l in file_list if "label" in l.split("/")[-1]]
#     for index in range(len(data_list)):
#         d=np.load(data_list[index])["arr_0"]
#         l=np.load(label_list[index])["arr_0"]
#         data=np.append(data,d,axis=0)
#         data=np.append(label,l,axis=0)
#     data=data[1:]
#     label=label[1:]
    for index in range(len(data_list)):
        data=np.load(data_list[index])["arr_0"]
        label=np.load(label_list[index])["arr_0"]
    return data,label

data,label=load_data(data_path)
print(data.shape)

(100, 20, 224, 224, 3)


In [5]:
print(keras.__version__)

2.2.4-tf


In [6]:
train_x, valid_x, train_y,valid_y =  train_test_split(data, label, test_size=0.1, random_state=42)
print(train_x.shape,valid_x.shape)

(90, 20, 224, 224, 3) (10, 20, 224, 224, 3)


In [7]:
# t_1,t_2,v_1,v_2=train_x.shape[0],train_x.shape[1],valid_x.shape[0],valid_x.shape[1]
# train_x.resize((t_1*t_2,img_size[0],img_size[1],3))
# valid_x.resize((v_1*v_2,img_size[0],img_size[1],3))

In [8]:
# train_y,valid_y=np.repeat(train_y,20),np.repeat(valid_y,20)
# print(train_y.shape)

In [9]:
# label_encoder = LabelEncoder()
# integer_encoded_t = label_encoder.fit_transform(train_y)
# integer_encoded_v = label_encoder.fit_transform(valid_y)
train_y.astype(int)
valid_y.astype(int)
# onehot_encoder = OneHotEncoder(sparse=False)
# train_y = train_y.reshape((len(train_y), 1))
# valid_y = valid_y.reshape((len(valid_y), 1))
# train_y = onehot_encoder.fit_transform(train_y)
# valid_y = onehot_encoder.fit_transform(valid_y)
print(valid_y.shape)

(10, 2)


In [10]:
base_model = keras.applications.InceptionV3(input_shape=img_size,
                                               include_top=False, 
                                               weights='imagenet')
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization_v1[0][0]     
_____________________________________

In [11]:
# base_model.trainable = False
# model = tf.keras.Sequential([
#     base_model,
#     #tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Reshape((49,512), input_shape=(7,7,512)),
#     #tf.keras.layers.Flatten(),
#     #tf.keras.layers.Dense(512, activation='relu'),
#     #tf.keras.layers.Dropout(0.3).
#     tf.keras.layers.LSTM(1024),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])
# model.summary()

In [12]:
#vgg16
# base_model.trainable = False
# model = Sequential()
# model.add(base_model)
# model.add(Reshape((16,512), input_shape=(4,4,512)))
# model.add(LSTM(1024,return_sequences=True))
# model.add(LSTM(256))
# #model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(10, activation='softmax'))

#     #tf.keras.layers.GlobalAveragePooling2D(),
# model.summary()

In [13]:
#inception
base_model.trainable = False
model = Sequential()
model.add(TimeDistributed(base_model,input_shape=(frame_number,img_size[0],img_size[1],img_size[2])))

model.add(TimeDistributed(GlobalAveragePooling2D()))
#model.add(Reshape((9,2048), input_shape=(3,3,2048)))
#model.summary()
#model.add(Reshape((2048)))
model.add(LSTM(128,return_sequences=True))
#,return_sequences=True
model.add(LSTM(64))
#model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_class, activation='softmax'))

    #tf.keras.layers.GlobalAveragePooling2D(),
model.summary()

TypeError: can only concatenate tuple (not "TensorShapeV1") to tuple

In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer="rmsprop",metrics=['accuracy'])

#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5',save_weights_only=True, period=10)
callbacks_list = [checkpoint]

model.fit(train_x, train_y, batch_size=3, epochs=30, validation_split=0.2,callbacks=callbacks_list)
#score = model.evaluate(valid_x, valid_y, batch_size=10)

In [ ]:
score = model.evaluate(valid_x, valid_y, batch_size=10)
print(score)

In [ ]:
base_model.trainable = False
test_model = Sequential()
test_model.add(TimeDistributed(base_model,input_shape=(frame_number,img_size[0],img_size[1],img_size[2])))
test_model.add(TimeDistributed(GlobalAveragePooling2D()))
test_model.add(LSTM(128,return_sequences=True))
test_model.add(LSTM(64))
test_model.add(Dense(32, activation='relu'))
test_model.add(Dropout(0.1))
test_model.add(Dense(num_class, activation='softmax'))
test_model.summary()
test_model.load_weights('weights00000009.h5')

In [ ]:
result=test_model.predict(valid_x,batch_size=10)
print(result)

In [ ]:
print(np.rint(result),valid_y)

In [ ]:
p=np.array([])
for i in range(len(result)):
    p.append(result[i].argmax())
print(p)